# ARMA Order Determination

In the previous notebook, the assumed model was ARMA(1,1)-GARCH(1,1). However, we should account for additional lags in the ARMA component of the model. To do this, I implement a brute-force search for p and q in ARMA(p,q) and choose the order that maximizes log-likelihood.

In [12]:
import numpy as np


def arma(c, phi, theta, r):
    T = len(r)
    epsilon = np.zeros(T)
    for t in range(T):
        if t < len(phi):
            epsilon[t] = r[t] - np.mean(r)
        else:
            ar_sum = np.sum([phi[i] * r[t-1-i] for i in range(len(phi))])
            ma_sum = np.sum([theta[i] * epsilon[t-1-i] for i in range(len(theta))])
            epsilon[t] = r[t] - c - ar_sum - ma_sum
    return epsilon


def garch(omega, alpha, beta, epsilon):
    T = len(epsilon)
    sigma_2 = np.zeros(T)
    for t in range(T):
        if t == 0:
            sigma_2[t] = omega / (1 - alpha - beta) # initialize as unconditional variance
        else:
            sigma_2[t] = omega + alpha*epsilon[t-1]**2 + beta*sigma_2[t-1]
    return sigma_2


def armagarch_negloglike(params, p, q, r):
    T = len(r)
    c = params[0] # (-10*mean, 10*mean)
    phi = params[1:p+1]
    theta = params[p+1:(p+q+2)] 
    omega = params[-3] # positive, (finfo.eps, 2*r.var())
    alpha = params[-2]
    beta = params[-1]
    
    epsilon = arma(c, phi, theta, r)
    sigma_2 = garch(omega, alpha, beta, epsilon)
    sigma_2 = np.where(sigma_2<=0, np.finfo(np.float64).eps, sigma_2)
    NegLogL = -0.5 * np.sum(-np.log(sigma_2) - epsilon**2/sigma_2)
    return NegLogL

In [26]:
from scipy.optimize import minimize

def determine_order(r, max_p, max_q):
    print('Running ARMA-GARCH Order Determination...')
    finfo = np.finfo(np.float64)
    temp_aic = np.inf
    import warnings
    warnings.filterwarnings("ignore")
    for q in range(1, max_q+1):
        for p in range(1, max_p+1):
                        
            # Define bounds for c, phi, theta, omega, alpha, beta
            c_bounds = [(-10*np.abs(np.mean(r)), 10*np.abs(np.mean(r)))]
            phi_bounds = [(-0.99999999, 0.99999999) for i in range(p)]
            theta_bounds = [(-0.99999999, 0.99999999) for i in range(q)]
            omega_bounds = [(finfo.eps, 2 * np.var(r))]
            alpha_bounds = [(finfo.eps, 0.99999999)]
            beta_bounds = [(finfo.eps, 0.99999999)]
            
            bounds = c_bounds + phi_bounds + theta_bounds + omega_bounds + alpha_bounds + beta_bounds

            initial_params =  tuple(0.0001 for _ in range(4+p+q))
            cons = (
                {'type': 'ineq', 'func': lambda x: x-finfo.eps},
                {'type': 'ineq', 'func': lambda x: 1-x[-1]-x[-2]+0.00000000000001}
            )
            res = minimize(armagarch_negloglike, initial_params, args=(p, q, r), bounds=bounds, method='SLSQP')
            neg_llh_val = res.fun
            aic = 2 * len(initial_params) + 2 * neg_llh_val
            if aic < temp_aic:
                best_p = p
                best_q = q
                temp_aic = aic
                print('Current best model: ARMA({},{})-GARCH(1,1), AIC = {}'.format(str(p), str(q), temp_aic))
                
    print('Order determination completed.')
    print('p =', best_p)
    print('q =', best_q)
    print('AIC =', temp_aic)

In [27]:
import pandas as pd
data = pd.read_csv('data/top10_logreturns.csv', index_col=0, parse_dates=True)['D05.SI']  # scaled for ease of optimization
determine_order(data.values, 6, 6)

Running ARMA-GARCH Order Determination...
Current best model: ARMA(1,1)-GARCH(1,1), AIC = -21589.13214933066
Current best model: ARMA(2,1)-GARCH(1,1), AIC = -21707.300379325097
Order determination completed.
p = 2
q = 1
AIC = -21707.300379325097
